In [1]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("huuuyeah/meetingbank")


c:\Users\Nik\anaconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = ds['train']
test_data = ds['test']
val_data = ds['validation']

ds

DatasetDict({
    train: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 5169
    })
    validation: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 861
    })
    test: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 862
    })
})

In [3]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
val_df = pd.DataFrame(val_data)

In [4]:
train_df.head()

,summary,uid,id,transcript
0,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_05012017_17-0161,0,Please refrain from profane or obscene speech....
1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...


In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [6]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
from langchain.agents import initialize_agent, load_tools
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

In [8]:
# dropping entries with too many tokens for the model
train_df_sorted = train_df[train_df['transcript'].str.len() <= 16385 * 4].reset_index()
train_df_sorted

,index,summary,uid,id,transcript
0,1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
1,2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
2,3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
3,4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...
4,5,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02212017_17-0161,5,"11 Eyes, Resolution 109 has been adopted as am..."
...,...,...,...,...,...
4882,5164,AN ORDINANCE imposing a tax on engaging in the...,SeattleCityCouncil_06052017_CB 118965,5164,Thank you. The bill passed and chair of the Se...
4883,5165,Proclamation Declaring April 8 through April 1...,AlamedaCC_04022019_2019-6703,5165,Okay. Next is a proclamation declaring April e...
4884,5166,Approves an agreement with Kaiser Foundation H...,DenverCityCouncil_01052015_14-1062,5166,I just pulled this one out. This one actually ...
4885,5167,A bill for an Ordinance authorizing and approv...,DenverCityCouncil_06082015_15-0302,5167,The estimated taxable value following redevelo...


In [9]:
train_df_sorted['transcript'][2]

"Motion passes. Hey, thank you very much. Now we're moving on to item 22. Communication from Vice Mayor Richardson recommendation to request City Council approval of the 2017 federal legislative agenda as recommended by the Federal Legislation Committee. There's a motion and a second Vice Mayor. So just a few updates here we have. So we met in November on November 15th to review staff's recommended recommended changes for the 2017 federal legislative agenda. The committee made some fairly substantive changes to the agenda last year when we were reorganized and consolidated some of the statements that may have been repetitive in years past. And because we made so many, so many changes in the last update, and as the new presidential administration gets up and running, the first large committee will be focused on protecting existing revenues that the city current receive currently receives from the federal government, such as the Housing Choice Voucher Program, commonly known as Section e

In [117]:
#audio transcription bot
from openai import OpenAI

client = OpenAI()
audio_file = open("Discuss RCM _ CPQ Data Usecases-20250411_081350-Meeting Recording.mp3", 'rb')

transcribed_audio = client.audio.transcriptions.create(
    file=audio_file,
    model='whisper-1',
    response_format='text'
)
transcribed_audio = '"' + transcribed_audio + '"'
print(transcribed_audio)

APIConnectionError: Connection error.

In [114]:
len(transcribed_audio)

17053

In [118]:
transcribed_audio = '''Chris Kennedy-Sloane: RCM and CPQ very firmly and I can accept that VPP has a management element of it, a price management element and potentially a customer price management element of it.
Chris Kennedy-Sloane: But I think that deal management is so poor that trying to involve it in any of these contexts doesn't make sense, right?
Israel Rodrigo: Well.
Chris Kennedy-Sloane: I'm almost like, can we just throw that away and stop selling it, please? Because it doesn't work.
Israel Rodrigo: Yeah. No, I know.
Israel Rodrigo: For me it was more think about buy side rebates and chargeback process between suppliers and distributors that have nothing to do with end customer or deal management.
Chris Kennedy-Sloane: Yes.
Israel Rodrigo: And still we need to be able to have the ability to capture, you know how those incentives or rebate programs are, you know, stored, handled, managed, etc.
Chris Kennedy-Sloane: I mean certainly for net, net and net net views of things like customer pricing.
Chris Kennedy-Sloane: I do agree, Israel, and that's been a consistent request, you know, and I don't want to solutionize too much because I appreciate we're at the top of the context conversation.
Chris Kennedy-Sloane: But I think from an RCM perspective, the critical point is that we need to have an output that.
Chris Kennedy-Sloane: Says. What is the value per SKU per customer that?
Chris Kennedy-Sloane: 'S either being given to us or being given away, so a true net net can be calculated.
Chris Kennedy-Sloane: That's like the number one requirement.
Chris Kennedy-Sloane: There's all these other ones on top of it.
Chris Kennedy-Sloane: But in the end, what everyone really wants to see is a true net net.
Chris Kennedy-Sloane: Sometimes they call it a net net net of all of the promotional activity that's going on around them. That's including rebates, that's including promotional activity.
Chris Kennedy-Sloane: So when they quote to a customer in a CPQ, they have a full waterfall, right?
Chris Kennedy-Sloane: I think that's why we've been directed here.
Chris Kennedy-Sloane: Anyone anyone disagree with that?
Israel Rodrigo: Yeah.
Chris Kennedy-Sloane: Is that fair?
Israel Rodrigo: Well, yeah, I I don't think it's a priority.
David Anderson: I mean, it's the priority for sure, yeah.
Israel Rodrigo: That's fine.
Israel Rodrigo: I'm just saying that again.
Israel Rodrigo: We want to have a comprehensive view of use cases to be covered.
Israel Rodrigo: Let's not dismiss the importance and relevance of buy side dynamics between cost management. Let's say on the distribution side, right, we have customer like ABC Supply. If eventually any of the pharma distributors want to consider like.
Israel Rodrigo: A RCM product.
Israel Rodrigo: And move away from EPP.
Israel Rodrigo: Right. Or at integration between RCM and MPP?
Israel Rodrigo: This could be also relevant.
Chris Kennedy-Sloane: That's a fair challenge. Israel. That's fair.
Israel Rodrigo: Zen.
Zain Master: OK, for the stupid person in the room.
Zain Master: Help me understand some of the terminology that's being thrown around because.
Zain Master: One I understand CPQ is responsible for discounting of the price from what the list price is.
Zain Master: That is different.
Chris Kennedy-Sloane: Not necessarily.
Chris Kennedy-Sloane: No, that's too much of A simplification saying I'm afraid. Sorry.
Zain Master: OK, no, no, go ahead. Correct me because I thought we in the in
Israel Rodrigo: Mm hmm.
Zain Master: the waterfall process are responsible for discounting a list price.
Zain Master: In some cases, we may or add to the list price as well and sell over cost like a car dealership as an example.
Zain Master: But rebates has incentives, right?
Zain Master: That's the right language in lingo. Is. Is that what it's called or?
David Anderson: What depends sometimes is just it's called a pro promotional program.
David Anderson: You know, customer programs can be like product specific or customer specific.
David Anderson: So you know, there's a little especially at Dell.
David Anderson: We see, you know, kind of even moving upstream into the price point process these discussions. But I think the point is you need to think Zain, I will say Zain, Zain.
David Anderson: Holistically about.
David Anderson: The you know the customer.
David Anderson: Gross to net.
David Anderson: So there definitely is a customer specific list price starting point,
Zain Master: OK.'''

In [106]:
def trim_string(text, max_length, placeholder="..."):
  """
    Trims a string to a maximum length, adding a placeholder if necessary.

    Args:
      text: The string to trim.
      max_length: The maximum length of the string.
      placeholder: The string to add if the text is trimmed.

    Returns:
      The trimmed string.
  """
  if len(text) <= max_length:
    return text
  else:
    return text[:max_length - len(placeholder)] + placeholder

In [111]:
transcribed_audio_trimmed = ''
if len(transcribed_audio) > 40000:
    transcribed_audio_trimmed = trim_string(transcribed_audio, 37000)

transcribed_audio_trimmed

''

In [ ]:
len(transcribed_audio)

40158

# the following models will take in either the transcribed audio, or a transcribed text document!

In [119]:
#summary bot

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4-turbo', temperature=0)


prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a detailed summary of the following:\\n\\n{text}")]
)

# Create Document objects from the filtered transcripts
docs = [Document(page_content=text) for text in transcribed_audio]

# Instantiate chain
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

# Invoke chain
try:
    result = chain.invoke({"input_documents": docs})
    print(result['output_text'])
except Exception as e:
    print(f"An error occurred: {e}")

The conversation involves Chris Kennedy-Sloane, Israel Rodrigo, and others discussing various aspects of business management, particularly focusing on pricing, rebates, and customer management. Chris Kennedy-Sloane initially acknowledges the management elements involved in VPP, including price management and customer price management. However, Chris expresses dissatisfaction with the current state of deal management, suggesting it is ineffective and proposing to discard it altogether.

Israel Rodrigo introduces the topic of buy-side rebates and chargeback processes between suppliers and distributors, which he notes do not involve end customers or deal management. This leads to a broader discussion about the need for a comprehensive view of incentives and rebate programs, and how they are managed and stored.

Chris Kennedy-Sloane emphasizes the importance of understanding the net pricing, including all promotional activities, to calculate a true "net net" price. The discussion highlight

In [120]:
# Call to action bot
prompt2 = """You are an intelligent assistant tasked with extracting all unique calls to action from the following text. Return each call to action seperately with a newline character. For example:

-Call to Action 1
-Call to Action 2
-Call to Action 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt2},
              {'role': 'user', 'content': transcribed_audio}]
)

calls = chain3.choices[0].message.content
print(calls)

- Stop selling deal management
- Let's not dismiss the importance and relevance of buy side dynamics
- We want to have a comprehensive view of use cases to be covered
- Help me understand some of the terminology that's being thrown around
- Correct me because I thought we in the waterfall process are responsible for discounting a list price
- Rebates has incentives, right?


In [13]:
'''# call to action bot

llm2 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.05)


prompt2 = ChatPromptTemplate.from_messages(
    [("system", "Return calls to action in bullet point form, of the following:\\n\\n{text}")]
)

# Create Document objects from the filtered transcripts
docs = [Document(page_content=text) for text in transcribed_audio]

# Instantiate chain
chain2 = load_summarize_chain(llm, chain_type="stuff", prompt=prompt2)

# Invoke chain
try:
    result2 = chain2.invoke({"input_documents": docs})
    print(result2['output_text'])
except Exception as e:
    print(f"An error occurred: {e}")'''

'# call to action bot\n\nllm2 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.05)\n\n\nprompt2 = ChatPromptTemplate.from_messages(\n    [("system", "Return calls to action in bullet point form, of the following:\\n\\n{text}")]\n)\n\n# Create Document objects from the filtered transcripts\ndocs = [Document(page_content=text) for text in transcribed_audio]\n\n# Instantiate chain\nchain2 = load_summarize_chain(llm, chain_type="stuff", prompt=prompt2)\n\n# Invoke chain\ntry:\n    result2 = chain2.invoke({"input_documents": docs})\n    print(result2[\'output_text\'])\nexcept Exception as e:\n    print(f"An error occurred: {e}")'

In [121]:
meeting_results = {'summary': result['output_text'], 'action calls': calls}

meeting_results['summary'] = 'Summary:\n\n' + meeting_results['summary'] + '\n'
meeting_results['action calls'] = ('Calls to Action:\n' + meeting_results['action calls']).replace('\n', '\n\n')

print(meeting_results['summary'])
print(meeting_results['action calls'])

Summary:

The conversation involves Chris Kennedy-Sloane, Israel Rodrigo, and others discussing various aspects of business management, particularly focusing on pricing, rebates, and customer management. Chris Kennedy-Sloane initially acknowledges the management elements involved in VPP, including price management and customer price management. However, Chris expresses dissatisfaction with the current state of deal management, suggesting it is ineffective and proposing to discard it altogether.

Israel Rodrigo introduces the topic of buy-side rebates and chargeback processes between suppliers and distributors, which he notes do not involve end customers or deal management. This leads to a broader discussion about the need for a comprehensive view of incentives and rebate programs, and how they are managed and stored.

Chris Kennedy-Sloane emphasizes the importance of understanding the net pricing, including all promotional activities, to calculate a true "net net" price. The discussion

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA


results = [transcribed_audio, meeting_results['summary'], meeting_results['action calls']]

embeddings = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

docsearch = FAISS.from_texts(
    texts=results,
    embedding=embeddings
)

llm3 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0)

qa = RetrievalQA.from_chain_type(
    llm=llm3,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':1}),
    return_source_documents=False
)


query = 'What is the summary for this meeting?'

response = qa.invoke(query)
print('Summary:\n')
print(response['result'] + '\n')

query_cta = 'What are the calls to action?'

response_cta = qa.invoke(query_cta + ' Return output in bullet points')
print('Calls to Action:\n')
print(response_cta['result'].replace('\n', '\n\n') + '\n')


query_any = 'Can you provide more context on "supporting workforce programs"?'

response_any = qa.invoke(query_any)
print('Context Result:\n')
print(response_any['result'].replace('\n', '\n\n'))

Summary:

The meeting was held to review and approve the 2017 Federal Legislative Agenda recommended by the Federal Legislation Committee to the City Council. The committee focused on protecting existing revenues from the Federal Government, supporting workforce programs, mental health services, violence prevention programs, and homeless prevention. They voted to adopt staff-recommended changes, including updates due to new executive orders. The agenda also includes advocating for the city, securing funding for major projects, and protecting residents. The meeting ended with acknowledgments and thanks from council members, and the motion was passed.

Calls to Action

- Approve the 2017 Federal Legislative Agenda as recommended by the Federal Legislation Committee

- Support existing revenues from the Federal Government, such as the Housing Choice Voucher Program and other HUD programs

- Work on workforce programs and support mental health services

- Advocate for federal legislation t

In [28]:
# NER bot
prompt3 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will extract the unique first and last names from the following text and separate each name with a newline character.

Name 1
Name 2
Name 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt3},
              {'role': 'user', 'content': transcribed_audio}]
)

names = chain3.choices[0].message.content
print(names)

Vice Mayor Richardson
Councilmember Urie
Councilmember Gonzalez
Councilmember Saranga
Councilmember Ringo
Councilmember Pierce


In [29]:
names_list = names.split('\n')
names_list

['Vice Mayor Richardson',
 'Councilmember Urie',
 'Councilmember Gonzalez',
 'Councilmember Saranga',
 'Councilmember Ringo',
 'Councilmember Pierce']

In [30]:
# dummy emails
members_emails = ['vice.richardson@example.org',
'c.urie@gov-office.net',
'gonzalez.council@city-hall.gov',
's.saranga@district.us',
'council.ringo@local-authority.com',
'p.pierce@towncouncil.info']

In [31]:
members_info = pd.DataFrame({'member': names_list, 'email': members_emails})
members_info

,member,email
0,Vice Mayor Richardson,vice.richardson@example.org
1,Councilmember Urie,c.urie@gov-office.net
2,Councilmember Gonzalez,gonzalez.council@city-hall.gov
3,Councilmember Saranga,s.saranga@district.us
4,Councilmember Ringo,council.ringo@local-authority.com
5,Councilmember Pierce,p.pierce@towncouncil.info


In [ ]:
# NER bot 2
prompt4 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will identify Unique calls to action and the names of the people the calls to action are directed towards. Your output should be like as follows:

Name 1: Call to Action
Name 2: Call to Action
Name 3: Call to Action
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt4},
              {'role': 'user', 'content': transcribed_audio}]
)

names_calls = chain3.choices[0].message.content
print(names_calls)

Vice Mayor Richardson: Request City Council approval of the 2017 Federal Legislative Agenda
Councilmember Gonzalez: Thank you to Diana and Vice Mayor Richardson 
Councilmember Saranga: Joining at the next federal legislation relation trip
Councilmember Ringo: Advocating for the city on behalf of Long Beach
Councilmember Pierce: Thanking for making sure the agenda is comprehensive and advocating for the community.


In [38]:
import re

delimiters = [r": ", "\n"]
pattern = "|".join(map(re.escape, delimiters))

names_calls_list = re.split(pattern, names_calls)
names_calls_list

['Vice Mayor Richardson',
 'Request City Council approval of the 2017 Federal Legislative Agenda',
 'Councilmember Gonzalez',
 'Thank you to Diana and Vice Mayor Richardson ',
 'Councilmember Saranga',
 'Joining at the next federal legislation relation trip',
 'Councilmember Ringo',
 'Advocating for the city on behalf of Long Beach',
 'Councilmember Pierce',
 'Thanking for making sure the agenda is comprehensive and advocating for the community.']

In [86]:
meeting_actions = {}
i = 0
while i < len(names_calls_list) - 1:
    name = names_calls_list[i].strip()
    action = names_calls_list[i + 1].strip()
    meeting_actions[name] = action
    i += 2

meeting_actions = pd.DataFrame.from_dict([meeting_actions]).T.reset_index().rename(columns={'index': 'member', 0: 'action'})
meeting_actions

,member,action
0,Vice Mayor Richardson,Request City Council approval of the 2017 Fede...
1,Councilmember Gonzalez,Thank you to Diana and Vice Mayor Richardson
2,Councilmember Saranga,Joining at the next federal legislation relati...
3,Councilmember Ringo,Advocating for the city on behalf of Long Beach
4,Councilmember Pierce,Thanking for making sure the agenda is compreh...


In [45]:
meeting_actions_emails = pd.merge(meeting_actions, members_info, on='member', how='outer').dropna()
meeting_actions_emails

,member,action,email
0,Councilmember Gonzalez,Thank you to Diana and Vice Mayor Richardson,gonzalez.council@city-hall.gov
1,Councilmember Pierce,Thanking for making sure the agenda is compreh...,p.pierce@towncouncil.info
2,Councilmember Ringo,Advocating for the city on behalf of Long Beach,council.ringo@local-authority.com
3,Councilmember Saranga,Joining at the next federal legislation relati...,s.saranga@district.us
5,Vice Mayor Richardson,Request City Council approval of the 2017 Fede...,vice.richardson@example.org


In [55]:
str(meeting_results)

'{\'summary\': "Summary:\\n\\nThe text is a transcript of a meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda as recommended by the Federal Legislation Committee to the City Council. The meeting took place on November 15th to review the recommended changes for the agenda. The committee made substantial changes to the agenda last year when they were reorganized and consolidated some statements that may have been repetitive in past years.\\n\\nThe focus of the committee is on protecting existing revenues that the City currently receives from the Federal Government, such as the Housing Choice Voucher Program (Section 8) and other HUD programs. They also plan to work on workforce programs and support mental health services. The committee also discussed adding language in the Public Safety section to support federal legislation that would assist in implementing violence prevention programs and homeless prevention.\\n\\nThe committee voted to

In [56]:
meeting_actions_emails_string = meeting_actions_emails.to_string(header=True, index=False)
meeting_results_string = str(meeting_results)

In [58]:
# Email bot
prompt5 = """You are an intelligent Email sending Assistant. You will take in a summary and calls to action from a meeting transcript, as well as a list of member names, with their corresponding specific action and Email address. Your task is to send the summary, calls to actions, and their corresponding specific action to each member's Email. An example Email Format is as follows:

To: sarah.chen@example.com

CC: project_team@example.com; management_team@example.com

From: david.lee@example.com

Subject: Project Phoenix - Kickoff Meeting Summary

Dear Sarah,

This email summarizes the kickoff meeting for Project Phoenix, held on October 26, 2023. The purpose of the meeting was to align the team on project goals, define key responsibilities, and establish the initial project timeline.

Meeting Summary:

The meeting began with an overview of Project Phoenix's objectives: to develop and launch a new customer relationship management (CRM) system within the next six months. We discussed the project's scope, which includes migrating existing customer data, integrating with our current marketing automation platform, and training our sales and support teams on the new system.

Key discussion points included:

- Data Migration Strategy: We reviewed the proposed approach for migrating customer data from the legacy system to the new CRM.

- Integration Requirements: We discussed the necessary integrations with the marketing automation platform, focusing on data synchronization and workflow automation.

- Training Plan: The training team outlined the plan for developing and delivering training materials to sales and support staff.

- Timeline and Milestones: We established the initial project timeline, identifying key milestones and deadlines. A detailed project schedule will be shared next week.

- Budget Allocation: Initial budget was presented.

Calls to Action:

- The following actions were assigned to ensure the project stays on track:

- Sarah Chen: Finalize the data migration plan and submit it for review by November 2nd.

- John Smith: Define the API specifications for the marketing automation platform integration by November 5th.

- Emily Brown: Develop the initial training modules outline by November 9th.

- David Lee: Circulate the detailed project schedule by October 28th.

- Michael Davis: Provide a detailed budget breakdown by November 2nd.

Specific Call to Action for Sarah Chen:

Sarah, your immediate action is to finalize the data migration plan, including the specific steps, resources required, and potential risks. Please submit this plan to project_team@example.com by November 2nd.

Thank you for your contributions to the meeting. Please let me know if you have any questions or require further clarification.

Best regards,

David Lee

Project Manager

Acme Corp

Here is the information:
"""

messages=[{'role': 'system', 'content': prompt5},
              {'role': 'user', 'content': [
                  {'type': 'text', 'text': meeting_results_string},
                  {'type': 'text', 'text': meeting_actions_emails_string}]}]

# Instantiate chain
try:
    chain4 = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages
    )

    email = chain4.choices[0].message.content
    print(email)
except Exception as e:
    print(f"An error occurred: {e}")

To: gonzalez.council@city-hall.gov

CC: project_team@example.com; management_team@example.com

From: vice.richardson@example.org

Subject: Approval of 2017 Federal Legislative Agenda

Dear Councilmember Gonzalez,

I hope this email finds you well. I am writing to follow up on the recent meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda to the City Council, as per the recommendation by the Federal Legislation Committee. The meeting, held on November 15th, focused on reviewing the proposed changes for the agenda, especially related to protecting existing revenues, workforce programs, mental health services, and public safety initiatives.

Meeting Summary:
The committee made substantial changes to the agenda last year during reorganization, aiming to consolidate statements and ensure the city's revenue interests with federal programs like the Housing Choice Voucher Program and HUD initiatives.

Action Calls:
Given your dedication to advoca

In [65]:
email_bones = []
for section in email.splitlines():
    email_bones.append(section)

email_bones

['To: gonzalez.council@city-hall.gov',
 '',
 'CC: project_team@example.com; management_team@example.com',
 '',
 'From: vice.richardson@example.org',
 '',
 'Subject: Approval of 2017 Federal Legislative Agenda',
 '',
 'Dear Councilmember Gonzalez,',
 '',
 'I hope this email finds you well. I am writing to follow up on the recent meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda to the City Council, as per the recommendation by the Federal Legislation Committee. The meeting, held on November 15th, focused on reviewing the proposed changes for the agenda, especially related to protecting existing revenues, workforce programs, mental health services, and public safety initiatives.',
 '',
 'Meeting Summary:',
 "The committee made substantial changes to the agenda last year during reorganization, aiming to consolidate statements and ensure the city's revenue interests with federal programs like the Housing Choice Voucher Program and HUD initi

In [79]:
to = email_bones[0]
cc = email_bones[2]
sender = email_bones[4]
subject = email_bones[6]
message_text = str(email_bones[8:]).replace("''", '\n').replace("',", '\n').replace("\n,", '').replace('\n', '\n\n').replace('",', '\n\n').replace('"', '').replace("'", "").replace('[', '\n').replace(']', '\n')


print(to)
print(cc)
print(sender)
print(subject)
print(message_text)


To: gonzalez.council@city-hall.gov
CC: project_team@example.com; management_team@example.com
From: vice.richardson@example.org
Subject: Approval of 2017 Federal Legislative Agenda

Dear Councilmember Gonzalez,

  I hope this email finds you well. I am writing to follow up on the recent meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda to the City Council, as per the recommendation by the Federal Legislation Committee. The meeting, held on November 15th, focused on reviewing the proposed changes for the agenda, especially related to protecting existing revenues, workforce programs, mental health services, and public safety initiatives.

  Meeting Summary:

 The committee made substantial changes to the agenda last year during reorganization, aiming to consolidate statements and ensure the citys revenue interests with federal programs like the Housing Choice Voucher Program and HUD initiatives.

  Action Calls:

 Given your dedication to 

In [ ]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build




